In [ ]:

import sys
#!{sys.executable} -m pip install xlrd
#!{sys.executable} -m pip install cmocean


In [ ]:
import xlrd
import numpy as np
%pylab inline
%matplotlib inline
import os
import glob
import h5py
import netCDF4

from scipy import signal
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.interpolate import griddata

import matplotlib as plot
import matplotlib.pyplot as plt

import cmocean as cmo
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

Plotting function

In [ ]:
def plotData(data,dExtent=None,ctrList=None,fsize=(10,10)):
    fig = plt.figure(figsize=fsize)
    ax1 = plt.gca()

    vmin = data.min()
    vmax = data.max()
    ax1.set_title('Depth (m)', fontsize=8)
    if dExtent is None:
        im1 = ax1.imshow(data,interpolation='nearest',
                         cmap=cmo.cm.ice_r,vmin=vmin, 
                         #extent = dExtent,
                         vmax=vmax)
    else:
        im1 = ax1.imshow(data,interpolation='nearest',
                         cmap=cmo.cm.ice_r,vmin=vmin, 
                         extent = dExtent,
                         vmax=vmax)
    
    if ctrList is not None:
        ax1.contour(data, ctrList, colors='w',  linewidths=2)
        
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("right", size="5%", pad=0.05)
    cbar1 = plt.colorbar(im1,cax=cax1)

    plt.tight_layout()
    plt.show()
    plt.close(fig)
    
    return

# Load the Excel dataset

In [ ]:
xyz = xlrd.open_workbook('One_tree_North_xyz.xlsx')
#xyz = xlrd.open_workbook('xyz_OT_windward.xlsx')
print xyz.sheet_names()

In [ ]:
xyz.sheet_by_name(u'Idealised_small') #.shape

In [ ]:
page1=xyz.sheet_by_name(u'Idealised_small')
#page1=xyz.sheet_by_name(u'xyz_OT_windward1')

## read in python

In [ ]:
x_ini=page1.col_values(0,start_rowx=1)
y_ini=page1.col_values(1,start_rowx=1)
#if (x_ini[0]==x_ini[1]):
#    x_ini=page1.col_values(1,start_rowx=1)
#    y_ini=page1.col_values(0,start_rowx=1)


z=page1.col_values(2,start_rowx=1)
nx_ini=len(x_ini)
ny_ini=len(y_ini)


nz=len(z)

x=[0]*nx_ini
y=[0]*ny_ini

for i in range (0,nx_ini):
    x[i]=abs(x_ini[i]-x_ini[0])

for i in range (0,ny_ini):
    y[i]=abs(y_ini[i]-y_ini[0])
    
nx=len(list(set(x)))
ny=len(list(set(y)))


take the list of points from excel and transform these points as a numpy array

In [ ]:
xarray = np.asarray(x) #.reshape((nx,ny))
yarray = np.asarray(y) #.reshape((nx,ny))
zarray = np.asarray(z) #.reshape((nx,ny))

pointsInput = np.vstack((xarray,yarray)).T

In [ ]:
zarray.shape

# Resample the dataset to desired dx

In [ ]:
# Resolution you want
dx = 5.

# MinX, maxX, minY, maxY
minX = xarray.min()
maxX = xarray.max()
minY = yarray.min()
maxY = yarray.max()

# Create mesh along X-Y
xi = np.arange(minX,maxX,dx)
yi = np.arange(minY,maxY,dx)
xmesh, ymesh = np.meshgrid(xi, yi)

Interpolate on finer grid from excel points

In [ ]:
newz = -griddata(pointsInput, zarray, (xmesh, ymesh), method='cubic')

Plotting function

In [ ]:
dExtent=[xi.min(),xi.max(),yi.min(),yi.max()]
ctrList = np.arange(1,40,5)
plotData(newz,None,ctrList)
plotData(newz,dExtent,None)

# Roger's function

In [ ]:
def rogeralpha(width,lamb):
    num = np.abs(np.cos(np.pi/2.*(1+width/lamb)))
    denum = 1.-num
    return num/denum

def rogerhx(x,nu,eps1,eps2):
    if x>=nu:
        eps = eps1
    else:
        eps = eps2
    return np.exp(-(x-nu)**2/(2.*eps**2)) 

def rogerhy(alpha,lamb,y):
    tmp = (1+alpha)*np.abs(np.cos(np.pi*y/lamb))-alpha
    return max(tmp,0)

def rogerSAG(hbase,hspr,hx,hy,tide):
    return hbase+hspr*hx*hy+tide

In [ ]:
hSAG = np.zeros((newz.shape))


In [ ]:
def typespag(num):
    if num==0:
        lamb = 4.   #Alongshore wavelength
        hspr = 1.75     #Spur height
        nu = 550.      #starting point of spur   
        eps1 = 61.   #Spur length
        eps2 = 10.     #how quickly it cuts off uphill of the starting point 
        width = 1.5   #groove width
        tide = 0.     #tidal level 

    if num==1:
        lamb =50.
        hspr = 0.
        nu = 600.
        eps1 = 77.
        eps2 = 20.
        width = 10.
        tide = 0.
    if num==2:
        lamb = 20.
        hspr = 2.9
        nu =550.
        eps1 = 77.
        eps2 = 20.
        width = 3.
        tide = 0.
    return lamb,hspr,nu,eps1,eps2,width,tide

In [ ]:
num=0
(lamb,hspr,nu,eps1,eps2,width,tide)=typespag(num)

alpha = rogeralpha(width,lamb)

for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
        hx = rogerhx(xmesh[j,i],nu,eps1,eps2)
        hy = rogerhy(alpha,lamb,ymesh[j,i])
        hSAG[j,i] = rogerSAG(newz[j,i],hspr,hx,hy,tide)
       


# Create the friction data list

In [ ]:
friction_min=0.03
friction_max=0.06
accuracy=0.01

#subtract the original bathy with the one wih the spurs and grooves to outline where they are.
h_fric= np.zeros((newz.shape))
for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
        if abs(hSAG[j,i]-newz[j,i])>accuracy:
            h_fric[j,i]=abs(hSAG[j,i]-newz[j,i])
            

h_friclist=[]
for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
        if h_fric[j,i]!=0.:
            h_friclist.append(h_fric[j,i])
h_unique=list(set(h_friclist))
h_unique_bis=sorted(h_unique)
len_h_unique=len(h_unique)



friction = np.arange(friction_min,friction_max,(friction_max-friction_min)/len_h_unique)

fric=np.ones((newz.shape))
fric=friction_min*fric
for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
        for l in range(0,len_h_unique):
            if h_fric[j,i]==h_unique_bis[l]:
                fric[j,i]=friction[l]        
        

In [ ]:
fric=np.ones((newz.shape))
fric=friction_max*fric
for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
        for l in range(0,len_h_unique):
            if h_fric[j,i]==h_unique_bis[l]:
                fric[j,i]=friction[l]
for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
            if fric[j,i]!=friction_max:
                fric[j,i]=friction_min
                

In [ ]:
plotData(hSAG,None,ctrList)
plotData(hSAG,dExtent,None)
plotData(fric,dExtent,None)
plotData(h_fric,dExtent,None)

In [ ]:
x = xmesh.T.flatten().tolist()
y = ymesh.T.flatten().tolist()
z = hSAG.T.flatten().tolist()
c_friction=fric.T.flatten().tolist()

nx=len(list(set(x)))
ny=len(list(set(y)))
nz=len(z)

In [ ]:
def grd(x,nx,ny,y):
    
    syst='Coordinate System = Cartesian'
    nx_str=str(nx)
    ny_str=str(ny)
    nx_ny=nx_str+' '+ny_str
    list_header=[syst,nx_ny,'0 0 0','']
    header="\n".join(list_header)
    
    if (x[0]==x[1]):
        len_list=ny*2+(nx+1)*2*ny
        xy_list=['0']*len_list
        
        for l in range (0,2*ny):
            xy_list[l*(nx+2)]="ETA"
            xy_list[1+l*(nx+2)]=str(l%ny+1)
        
        y_start=2+(nx+2)*(ny)
        for i in range (0,ny):
            for j in range (0,nx):
                xy_list[2+(nx+2)*i+j]=str(x[i+j*ny])
                xy_list[y_start+(nx+2)*i+j]=str(y[i+j*ny])
                
        
    else:     
        len_list=ny*2+(nx+1)*2*ny
        xy_list=['0']*len_list
        
        for l in range (0,2*ny):
            xy_list[l*(nx+2)]="ETA"
            xy_list[1+l*(nx+2)]=str(l%ny+1)
        
        y_start=2+(nx+2)*(ny)
        for i in range (0,ny):
            for j in range (0,nx):
                xy_list[2+(nx+2)*i+j]=str(x[j+i*nx])
                xy_list[y_start+(nx+2)*i+j]=str(y[j+i*nx])
                
                
                
    xy=" ".join(xy_list)
    file_complete=header+xy
    newfile= open("xy.grd", "w")
    newfile.write(file_complete)
    newfile.close()
    
    dfile = file('xy.grd','r')
    efile = file('xy2.grd','w')
    efile.write(dfile.read().replace('ETA', '\nETA'))
    
    
    
    return
        

In [ ]:
dfile = file('untitled.txt','r')
efile = file('untitled2.txt','w')
efile.write(dfile.read().replace(' ', '\n '))

In [ ]:
grd(x,nx,ny,y)

In [ ]:
def bed(x,z,nz,nx,ny):
    bed_list=[]
    reef_list=[]
    nx_value=''
    reef_nx_value=''
    
    if (x[0]==x[1]):
        for i in range(0,ny):
            for j in range (0,nx):
                #nx_value=nx_value+str(-z[i+j*ny])+' '
                nx_value=str(z[i+j*ny])+' '+nx_value
                reef_nx_value=reef_nx_value+'0'+' '
            bed_list=bed_list+[nx_value]
            reef_list=reef_list+[reef_nx_value]
            nx_value=''
            reef_nx_value=''  
        
        
        
    else:
        for j in range(0,ny):
            for i in range (0,nx):
                #nx_value=nx_value+str(-z[i+j*nx])+' '
                nx_value=str(z[i+j*ny])+' '+nx_value
                reef_nx_value=reef_nx_value+'0'+' '
            bed_list=bed_list+[nx_value]
            reef_list=reef_list+[reef_nx_value]
            nx_value=''
            reef_nx_value=''
     
    bed="\n".join(bed_list)
    reef="\n".join(reef_list)
    
    
    
    newfile= open("bed.dep", "w")
    newfile.write(bed)
    newfile.close()
    
    newfile= open("reef.dep", "w")
    newfile.write(reef)
    newfile.close()
    return

In [ ]:
bed(x,z,nz,nx,ny)

In [ ]:
#c_friction=[0.03]*nz

In [ ]:
def frictionc(c_friction,nx,ny):
    c_list=[]
    c_value=''
       
    for j in range(0,ny):
        for i in range (0,nx):
            c_value=str(c_friction[i+j*ny])+' '+c_value
        c_list=c_list+[c_value]
        c_value=''
    c="\n".join(c_list)
    newfile= open("fwfile.txt", "w")
    newfile.write(c)
    newfile.close()
    return

In [ ]:
frictionc(c_friction,nx,ny)

# Find the x and y coordinate for a maximum spur and a minimum groove

First, find the spurs coordinate in the x and y list. this outputs the x and the y coordinate which then are input into the next cell. c=h_fric[:,the y]. 

when using a + in the rogers function this cell outputs the location of the grooves
when using a - in the rogers function this cell outputs the location of the spurs

In [ ]:
for j in range(0,newz.shape[0]):
    for i in range(0,newz.shape[1]):
        if abs(hSAG[j,i]-newz[j,i])>accuracy:
            h_fric[j,i]=abs(hSAG[j,i]-newz[j,i])
a=np.max(h_fric)
np.where(h_fric==a)

Then, with the same coordinate in the x list, we find the grooves coordinate in the y list

when using a - in the rogers function this cell outputs the location of the grooves
when using a + in the rogers function this cell outputs the location of the spurs

In [ ]:
c=h_fric[:,110]
b=np.amin(c)
np.where(c==b)

Then, find the x and y coordinate in the mesh. this outputs the coordinates in metres 

In [ ]:
print yi[42],yi[44]

In [ ]:
points_max=900 # x coordinate for the last point. the values are inverted. ie the orgin is in the bottom right of the plt
points_min=500  # x coordinate for the first point
npoints=50
arr_points = np.arange(points_min,points_max,(points_max-points_min)/npoints)
points=arr_points.tolist()
#points[0]=300
point_list=[]
for i in range(0,npoints):
    point_value=str(points[i])+' '+'210'   #this value comes from the coordinate of the spurs in metres
    point_list=point_list+[point_value]   
for i in range(0,npoints):
    point_value=str(points[i])+' '+'220'   #this value comes from the coordinate of the grooves in metres
    point_list=point_list+[point_value]
p="\n".join(point_list)
newfile= open("point.txt", "w")
newfile.write(p)
newfile.close()

# this cell creates the input waves for the non-hydrostatic mode

In [ ]:
H=0.15 # amplitude of the sinusoidal u --> height of the wave
T=17 # period of the sinusoidal u --> The greater the period the larger the wave
T_wave=20 # period of the wave --> time between two waves
tstop=300
nonhyd=[]

#time = nc_data.variables['globaltime'][:]
time=np.arange(0,tstop,1)

u = H*np.sin(time*2*np.pi/T)+H
u_period=u[:T+1]
a=np.where(u_period==np.min(u_period))
b=a[0][0]
u_bis=u[b-1:b+T+1]
u_bis[0]=0.0

for j in range(0,tstop,size(u_bis)+T_wave):
    for i in range(0,size(u_bis)):
        value=str(time[i+j])+' '+str(u_bis[i])
        nonhyd=nonhyd+[value]
nhyd="\n".join(nonhyd)
newfile= open("Boun_U.bcf", "w")
newfile.write(nhyd)
newfile.close()

# this creates the numbers for the output, but you need to add this at the beginning of the file before giving it to XBeach.
#    scalar
#    2
#    t U


